In [ ]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")
MODEL    = os.getenv("MODEL_NAME", "bens_model")

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": 128,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


## Three of thought: Second inference method

In [ ]:
class Node:
    def __init__(self, thought, children = None):
        self.thought = thought
        self.children = children or []

class Tot:
    def __init__(self, prompt, model = None, iterations = 3, maxTokens = 128):
        self.root = Node(prompt)
        self.model = model or MODEL
        self.iterations = iterations
        self.current_thoughts = [self.root]
        self.maxTokens = maxTokens


    def EThoughts(self, thoughtN):
        newThoughtNodes = []

        for nd in thoughtN:
            prompt = f"Given the current thought: '{nd.thought}', provide three concise thoughts that expand this idea further."
            response = call_model_chat_completions(prompt)


            if response:
                child = Node(response["text"])
                nd.children.append(child)
                newThoughtNodes.append(child)

        return newThoughtNodes


    def Run(self):
        iteration = 0
        while self.current_thoughts and iteration < self.iterations:
            print(f"Iteration {iteration + 1}:")
            self.current_thoughts = self.EThoughts(self.current_thoughts)

            for tNode in self.current_thoughts:
                print(f"Explored Thought: {tNode.thought}")
            iteration += 1


    def updateThought(self, nThought):
        self.root = Node(nThought)
        self.current_thoughts = [self.root]


    def showTree(self, Node, lv=0, prefix=""):
        connector = "└── " if lv > 0 else ""
        print(f"{prefix}{connector}{Node.thought}")

        # Prepare indentation for children
        child_prefix = prefix + ("    " if lv > 0 else "")

        for child in Node.children:
            self.showTree(child, lv + 1, child_prefix)


if __name__ == "__main__":
    starting_prompt = "what is 2 + 2"
    Tot = Tot(starting_prompt)
    Tot.Run()
    print("=" * 100)
    print("Final Tree of Thoughts:")
    Tot.showTree(Tot.root)
